# コードなしのデプロイとカスタムコードのデプロイ

本ノートブックでは、
 - Azure Machine Learning ワークスペースを登録する方法を学びます。
 - Azure Container Instances にモデルをデプロイする2通りの方法を学びます。
     - コードなしのデプロイ
     - カスタムコードのデプロイ

※ 参照：[Azure Machine Learning を使用してモデルをデプロイする](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-and-where)

## Python SDK のインポート

In [1]:
import azureml.core

# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.0.76


## Azure Machine Learning ワークスペースへの接続

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

azureml-iot
iothack
southeastasia
9c0f91b8-eb2f-484c-979c-15848c098a6b


## インプットデータとアウトプットデータの指定

scikit-learn の [diabates データセット](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) を用いて学習された `sklearn_regression_model.pkl` を登録してきます。まず、このモデルへのインプットデータとアウトプットデータと指定します。

In [6]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files=['./features.csv', './labels.csv'],
                       target_path='sklearn_regression/',
                       overwrite=True)

input_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/features.csv')])
output_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'sklearn_regression/labels.csv')])

Uploading an estimated of 2 files
Uploading ./features.csv
Uploading ./labels.csv
Uploaded ./labels.csv, 1 files out of an estimated total of 2
Uploaded ./features.csv, 2 files out of an estimated total of 2
Uploaded 2 files


## モデルの登録

[Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-) を呼び出してモデル登録を行います。

また、モデルのフレームワークやバージョンなどのメタ情報も付与することで "コードなしのデプロイ" の機能を有効にします。

In [4]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model-no-data',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='0.19.1',             # Version of scikit-learn used to create the model.
                       #sample_input_dataset=input_dataset,
                       #sample_output_dataset=output_dataset,
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-sklearn-model-no-data
Name: my-sklearn-model-no-data
Version: 3


## モデルのデプロイ (ノーコード & カスタム)

[Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-) メソッドを用いてモデルをデプロイします。本ノートブックでは、テスト環境でよく使われる Azure Container Instances (ACI) にデプロイします。※ 本番環境では、Azure Kubernetes Service (AKS) の利用を推奨。

### ノーコードデプロイメント (preview)

In [9]:
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = 'aaa'

# 既存の推論環境で同じ名前のものを削除する
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

service = Model.deploy(ws, service_name, [model])
service.wait_for_deployment(show_output=True)

Running...................
Succeeded
ACI service creation operation finished, operation "Succeeded"


モデルのデプロイが完了したら、Web Service を Call して推論を行います。

In [11]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ],
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

KeyboardInterrupt: 

### カスタムコードのデプロイメント

全ての機械学習ライブラリに対応する方法です。コードなしのデプロイとは違って推論スクリプトや Python ライブラリなどの情報が必要になります。

Python のライブラリなどの情報は [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) で管理します。

※ 参考:[Azure Machine Learning を使用してトレーニングとデプロイのための環境を再利用します。](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-use-environments)

In [12]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'scikit-learn'
])

カスタムコードのデプロイにおいては、推論用の Python スクリプトが必須になります。

In [13]:
with open('score.py') as f:
    print(f.read())

import joblib
import numpy as np
import os

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType


def init():
    global model

    model_filename = 'sklearn_regression_model.pkl'
    model_path = os.path.join(os.environ['AZUREML_MODEL_DIR'], model_filename)
    model = joblib.load(model_path)


@input_schema('data', NumpyParameterType(np.array([[0.1, 1.2, 2.3, 3.4, 4.5, 5.6, 6.7, 7.8, 8.9, 9.0]])))
@output_schema(NumpyParameterType(np.array([4429.929236457418])))
def run(data):
    # Use the model object loaded by init().
    result = model.predict(data)

    # You can return any JSON-serializable object.
    return result.tolist()



[InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) オブジェクトを[Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-)渡して、カスタムコードのデプロイメントを行います。

In [15]:
from azureml.core import Webservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

service_name = 'my-custom-env-service'

# 既存の推論環境で同じ名前のものを削除する
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py',
                                   source_directory='.',
                                   environment=environment)

service = Model.deploy(ws, service_name, [model], inference_config)
service.wait_for_deployment(show_output=True)

Running.........................
Succeeded
ACI service creation operation finished, operation "Succeeded"


デプロイが完了したら、 [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-) を用いて推論を行います。

In [18]:
import json

input_payload = json.dumps({
    'data': [
        [ 0.03807591,  0.05068012,  0.06169621, 0.02187235, -0.0442235,
         -0.03482076, -0.04340085, -0.00259226, 0.01990842, -0.01764613]
    ]
})

output = service.run(input_payload)

print(output)

[193.20580440332455]


In [17]:
# Swagger ファイル (json) の取得
service.swagger_uri

'http://87dc50a9-28b0-4243-8fab-5c8d8cf8746f.southeastasia.azurecontainer.io/swagger.json'

### モデルのプロファイリング (Model profiling)

必要な CPU と Memory のスペックをプロファイルすることもできます。

```python
profile = Model.profile(ws, "profilename", [model], inference_config, test_sample)
profile.wait_for_profiling(True)
profiling_results = profile.get_results()
print(profiling_results)
```

### モデルのパッケージ化 (Model packaging)

推論環境 を Docker イメージにパッケージ化することも可能です。Docker イメージは ACR に格納されます。

`Model.package()` に Environment オブジェクトを渡す必要があります。

```python
package = Model.package(ws, [model], inference_config)
package.wait_for_creation(show_output=True)  # Or show_output=False to hide the Docker build logs.
package.pull()
```

Docker イメージをローカルでビルドするために必要な Dockerfile, モデル, およびその他のアセットをダウンロードすることもできます。

```python
package = Model.package(ws, [model], inference_config, generate_dockerfile=True)
package.wait_for_creation(show_output=True)
package.save("./local_context_dir")
```